In [1]:
import ipywidgets as widgets
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import Image
from IPython.core.display import HTML 
import time
from ipywidgets import FloatProgress
from IPython.display import display
import pafy
from bs4 import BeautifulSoup
import requests
import re
from IPython.display import HTML
import numpy as np
import pandas as pd
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from IPython.display import Image

In [2]:
from newsapi import NewsApiClient
newsapi = NewsApiClient(api_key='a539d7df2c7b43e1ac4d12f386d901e8')

In [48]:
search_parameter = 'brett kavanaugh'
sources_list = ['abc-news',
 'associated-press',
 'axios',
 'bbc-news',
 'bbc-sport',
 'bleacher-report',
 'bloomberg',
 'breitbart-news',
 'business-insider',
 'buzzfeed',
 'cbc-news',
 'cbs-news',
 'cnbc',
 'cnn',
 'crypto-coins-news',
 'daily-mail',
 'engadget',
 'entertainment-weekly',
 'espn',
 'financial-post',
 'financial-times',
 'fox-news',
 'fox-sports',
 'google-news',
 'hacker-news',
 'ign',
 'independent',
 'mashable',
 'medical-news-today',
 'msnbc',
 'mtv-news',
 'national-geographic',
 'national-review',
 'nbc-news',
 'new-scientist',
 'newsweek',
 'new-york-magazine',
 'nfl-news',
 'nhl-news',
 'nrk',
 'politico',
 'recode',
 'reddit-r-all',
 'reuters',
 'techcrunch',
 'techradar',
 'the-american-conservative',
 'the-economist',
 'the-guardian-au',
 'the-guardian-uk',
 'the-huffington-post',
 'the-lad-bible',
 'the-new-york-times',
 'the-next-web',
 'the-sport-bible',
 'the-telegraph',
 'the-verge',
 'the-wall-street-journal',
 'the-washington-post',
 'the-washington-times',
 'time',
 'usa-today',
 'vice-news',
 'wired',
 'ynet']
sources_joined = ','.join(sources_list)

In [49]:
def counter_results(used_sources, approved_sources):
    new_sources = []
    for item in approved_sources:
        if item in used_sources:
            pass
        else:
            new_sources.append(item)
    new_sources = ','.join(new_sources)
    return new_sources

In [50]:
def split_source_info(list_of_dicts):
    for item in list_of_dicts:
        item['source_id'] = item['source']['id']
        item['source_name'] = item['source']['name']

In [51]:
def pull_videos(parameter):
    list_dicts = []
    split = parameter.split()
    if len(split) < 2:
        split.append('')
    html_page = requests.get('https://www.youtube.com/results?search_query='+split[0]+'+'+'+'+split[1])
    soup = BeautifulSoup(html_page.content, 'html.parser', from_encoding='utf-8')
    videos = soup.findAll('a')
    videos = videos[40:]
    for video in videos:
        if video.get('href')[:9] == '/watch?v=':
            url = "https://www.youtube.com" + video.get('href')
            video = pafy.new(url)
            data = {'author':video.author, 'content':video.category,'description':video.description,'publishedAt':video.published, 'source_id':video.username, 'source_name':video.author,'title':video.title,'url':url,'urlToImage':video.thumb, 'medium':'video'}
            list_dicts.append(data)
    return list_dicts

In [52]:
def pull_pods(parameter):
    list_dicts = []
    urls = []
    split = parameter.split()
    if len(split) < 2:
        split.append('')
    html_page = requests.get('https://tunein.com/search/?query='+split[0]+'%'+'20'+split[1])
    soup = BeautifulSoup(html_page.content, 'html.parser', from_encoding='utf-8')
    pods = soup.findAll('a')
    strings = []
    for number in list(range(0,10)):
        strings.append(str(number))
    for pod in pods[6:]:
        if pod.get('href') is not None:
            if pod.get('href')[-1] in strings:
                url = "https://tunein.com/embed/player/t"+pod.get('href')[-9:]+"/"
                data = {'author':'audio', 'content':'audio','description':'audio','publishedAt':'audio', 'source_id':'audio', 'source_name':'audio','title':'audio','url':url,'urlToImage':'audio', 'medium':'audio'}
                if url in urls:
                    pass
                else:
                    list_dicts.append(data)
                urls.append(url)
    return list_dicts

In [53]:
search_widget = widgets.Text(placeholder = 'Search News', description='Search:', disabled=False)

def search_news(parameter):
    if len(parameter) > 0:
        time.sleep(5)
        print('Aggregating...')
        article_results_rel = newsapi.get_everything(q=parameter,sort_by = 'relevancy',language='en', page_size=50, sources=sources_joined)
        sources = list(set([article['source']['id'] for article in article_results_rel['articles']]))
        opposing_sources = counter_results(sources, sources_list)
        article_results_opp = newsapi.get_everything(q=parameter,sort_by = 'relevancy',language='en', page_size=50, sources=opposing_sources)
        article_results_rel = article_results_rel['articles']
        article_results_opp = article_results_opp['articles']
        print('Pulling videos...')
        video_df = pd.DataFrame(pull_videos(parameter))
        print('Pulling audio...')
        audio_df = pd.DataFrame(pull_pods(parameter))
        consolidated = article_results_opp + article_results_rel
        split_source_info(consolidated)
        df = pd.DataFrame(consolidated)
        df = df.drop('source',axis =1)
        df['medium'] = 'text'
        df = df.append(video_df, ignore_index=True)
        df = df.append(audio_df, ignore_index=True)
        display(df[(df.medium == 'video')])
        #temporary fillers
        sample_size = len(list(df.medium))
        random_polarity = np.random.random_sample(sample_size)
        random_article_similarity =np.random.random_sample(sample_size)
        random_source_similarity =np.random.random_sample(sample_size)
        random_article_lengths = np.random.randint(1,20,size=sample_size)
        random_cosine_similarity = np.random.random_sample(sample_size)
        random_subject = np.random.randint(1,10,size=sample_size)
        random_cluster = np.random.randint(1,4,size=sample_size)
        df['polarity'] = random_polarity
        df['length'] = random_article_lengths
        df['article_similarity'] = random_article_similarity
        df['source_similarity'] = random_source_similarity
        df['cosine_similarity'] = random_cosine_similarity
        df['subject_matter'] = random_subject
        df['cluster'] = random_cluster
        df.to_csv('current_search.csv')
        time.sleep(5)
        print('Data Loaded!')
#         time.sleep(1800)
    else:
        print('Enter Search Parameter')

interact(search_news, parameter=search_widget);

interactive(children=(Text(value='', description='Search:', placeholder='Search News'), Output()), _dom_classe…

In [55]:
# search_news('Brett Kavanaugh')

# Perspective 1

In [15]:
# pd.read_csv('current_search.csv')

In [56]:
df = pd.read_csv('current_search.csv', index_col=0)
Length_widget1 = widgets.IntRangeSlider(min=df.length.min(), max=df.length.max(), step=1, value = [1,19])
Perspective_widget1 = widgets.Dropdown(options=set(list(df.cluster)),value = 1,description = 'Perspective', disabled = False)
Limit_widget1 = widgets.IntSlider(min=1, max=20, step=1, value=1)
Medium_widget1 = widgets.ToggleButtons(options = ['Text', 'Audio','Video'], description ='Medium', button_style='')

def pull_articles(Length, Perspective, Limit, Medium):
    df = pd.read_csv('current_search.csv', index_col=0)
    df = df.dropna()
    if Medium == 'Text':
        df2 = df[(df.length > Length[0]) & (df.length < Length[1]) & (df.cluster == Perspective) & (df.medium == 'text')]
        list_tuples = []
        for i in range (0, Limit):
            title = list(df2.title)[i]
            link = list(df2.url)[i]
            image = list(df2.urlToImage)[i]
            source_name = list(df2.source_name)[i]
            display(HTML("<a href="+link+">"+source_name+': '+title+"</a>"))
            display(Image(url= image))
    elif Medium == 'Video':
        df2 = df[(df.length > Length[0]) & (df.length < Length[1]) & (df.cluster == Perspective) & (df.medium == 'video')]
        list_tuples = []
        for i in range (0, Limit):
            title = list(df2.title)[i]
            link = list(df2.url)[i]
            link = link[-11:]
            source_name = list(df2.source_name)[i]
            display(HTML("<a href="+link+">"+source_name+': '+title+"</a>"))
            display(HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/'+link+'?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>'))
    else:
        df2 = df[(df.length > Length[0]) & (df.length < Length[1]) & (df.cluster == Perspective) & (df.medium == 'audio')]
        list_tuples = []
        if len(list(df2.index)) < 1:
            print('No Audio')
        else:
            for i in range (0, Limit):
                link = list(df2.url)[i]
                display(HTML("<iframe src="+"'"+link+"'"+ "style='width:100%; height:100px;' scrolling='no' frameborder='no'></iframe>"))




interact(pull_articles,Length=Length_widget1, Perspective=Perspective_widget1, Limit = Limit_widget1, Medium = Medium_widget1);

interactive(children=(IntRangeSlider(value=(1, 19), description='Length', max=19, min=1), Dropdown(description…

# Perspective 2

In [57]:
Length_widget2 = widgets.IntRangeSlider(min=df.length.min(), max=df.length.max(), step=1, value = [1,19])
Perspective_widget2 = widgets.Dropdown(options=set(list(df.cluster)),value = 2,description = 'Perspective', disabled = False)
Limit_widget2 = widgets.IntSlider(min=1, max=20, step=1, value=1)
Medium_widget2 = widgets.ToggleButtons(options = ['Text', 'Audio','Video'], description ='Medium', button_style='')

def pull_articles(Length, Perspective, Limit, Medium):
    df = pd.read_csv('current_search.csv', index_col=0)
    df = df.dropna()
    if Medium == 'Text':
        df2 = df[(df.length > Length[0]) & (df.length < Length[1]) & (df.cluster == Perspective) & (df.medium == 'text')]
        list_tuples = []
        for i in range (0, Limit):
            title = list(df2.title)[i]
            link = list(df2.url)[i]
            image = list(df2.urlToImage)[i]
            source_name = list(df2.source_name)[i]
            display(HTML("<a href="+link+">"+source_name+': '+title+"</a>"))
            display(Image(url= image))
    elif Medium == 'Video':
        df2 = df[(df.length > Length[0]) & (df.length < Length[1]) & (df.cluster == Perspective) & (df.medium == 'video')]
        list_tuples = []
        for i in range (0, Limit):
            title = list(df2.title)[i]
            link = list(df2.url)[i]
            link = link[-11:]
            source_name = list(df2.source_name)[i]
            display(HTML("<a href="+link+">"+source_name+': '+title+"</a>"))
            display(HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/'+link+'?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>'))
    else:
        df2 = df[(df.length > Length[0]) & (df.length < Length[1]) & (df.cluster == Perspective) & (df.medium == 'audio')]
        list_tuples = []
        if len(list(df2.index)) < 1:
            print('No Audio')
        else:
            for i in range (0, Limit):
                link = list(df2.url)[i]
                display(HTML("<iframe src="+"'"+link+"'"+ "style='width:100%; height:100px;' scrolling='no' frameborder='no'></iframe>"))


interact(pull_articles,Length=Length_widget2, Perspective=Perspective_widget2, Limit = Limit_widget2, Medium = Medium_widget2);

interactive(children=(IntRangeSlider(value=(1, 19), description='Length', max=19, min=1), Dropdown(description…

# Perspective 3

In [58]:
Length_widget3 = widgets.IntRangeSlider(min=df.length.min(), max=df.length.max(), step=1, value = [1,19])
Perspective_widget3 = widgets.Dropdown(options=set(list(df.cluster)),value = 3,description = 'Perspective', disabled = False)
Limit_widget3 = widgets.IntSlider(min=1, max=20, step=1, value=1)
Medium_widget3 = widgets.ToggleButtons(options = ['Text', 'Audio','Video'], description ='Medium', button_style='')

def pull_articles(Length, Perspective, Limit, Medium):
    df = pd.read_csv('current_search.csv', index_col=0)
    df = df.dropna()
    if Medium == 'Text':
        df2 = df[(df.length > Length[0]) & (df.length < Length[1]) & (df.cluster == Perspective) & (df.medium == 'text')]
        list_tuples = []
        for i in range (0, Limit):
            title = list(df2.title)[i]
            link = list(df2.url)[i]
            image = list(df2.urlToImage)[i]
            source_name = list(df2.source_name)[i]
            display(HTML("<a href="+link+">"+source_name+': '+title+"</a>"))
            display(Image(url= image))
    elif Medium == 'Video':
        df2 = df[(df.length > Length[0]) & (df.length < Length[1]) & (df.cluster == Perspective) & (df.medium == 'video')]
        list_tuples = []
        for i in range (0, Limit):
            title = list(df2.title)[i]
            link = list(df2.url)[i]
            link = link[-11:]
            source_name = list(df2.source_name)[i]
            display(HTML("<a href="+link+">"+source_name+': '+title+"</a>"))
            display(HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/'+link+'?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>'))
    else:
        df2 = df[(df.length > Length[0]) & (df.length < Length[1]) & (df.cluster == Perspective) & (df.medium == 'audio')]
        list_tuples = []
        if len(list(df2.index)) < 1:
            print('No Audio')
        else:
            for i in range (0, Limit):
                link = list(df2.url)[i]
                display(HTML("<iframe src="+"'"+link+"'"+ "style='width:100%; height:100px;' scrolling='no' frameborder='no'></iframe>"))



interact(pull_articles,Length=Length_widget3, Perspective=Perspective_widget3, Limit = Limit_widget3, Medium = Medium_widget3);

interactive(children=(IntRangeSlider(value=(1, 19), description='Length', max=19, min=1), Dropdown(description…

# Perspective Metrics

In [33]:
df2 = df[(df.cluster == 1)]

In [14]:
all_words = pd.read_csv('word_metrics.csv')
final_df = pd.read_csv('bk_corpus.csv')
all_words['word']= all_words['word'].astype(str)
subjectivity_widget = widgets.FloatSlider(min=0, max=1, step=.1, value = .8)
source_widget = widgets.Dropdown(options=set(list(final_df.source_name)),value = 'The New York Times',description = 'Source', disabled = False)


def word_cloud(source_name,subjectivity_floor):
    
    list_words = all_words.word[(all_words.subjectivity < subjectivity_floor)]
    df2 = final_df[(final_df.source_name == source_name)]
    all_text = []
    for blob in df2.text:
        all_text.append(blob)
    corpus = '-'.join(all_text)
    corpus = corpus.lower()
    stopwords = set(STOPWORDS)
    stopwords.update(list_words)

    # Generate a word cloud image
    wordcloud = WordCloud(background_color="white", stopwords=stopwords).generate(corpus)
    # Display the generated image:
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()
interact(word_cloud,source_name = source_widget, subjectivity_floor = subjectivity_widget);

interactive(children=(Dropdown(description='Source', index=8, options=('The Washington Post', 'Mashable', 'Reu…